## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

# Решение

### 1. Обработать данные, получить токены, обработать токены.

In [108]:
import pymorphy2
import nltk
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, CategoricalNB


#nltk.download('punkt')
#nltk.download('stopwords')


data = []
morph = pymorphy2.MorphAnalyzer()

# Read Text
with open("data/news_train.txt", 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        vec = []
        category, topic, content = line.split("\t")
        vec.append(category)
        vec.append(topic)
        vec.append(content)
        data.append(vec)

# Tokenize

data_tokens = []

for vec in data:
    vec_tokenize = [vec[0]]
    sentences = sent_tokenize(vec[1])
    vec_tokenize.append([word_tokenize(sent) for sent in sentences])
    sentences = sent_tokenize(vec[1])
    vec_tokenize.append([word_tokenize(sent) for sent in sentences])
    data_tokens.append(vec_tokenize)

# Process tokens

counter = 0
token_limit = 5


for vec in data_tokens:
    for i in range(1,2):
        sentence = vec[i]
        for tokens in sentence:
            for token in tokens:
                print(morph.parse(token))
                counter += 1
                if counter > token_limit:
                    break
            if counter > token_limit:
                break
        if counter > token_limit:
            break
    if counter > token_limit:
        break


[Parse(word='овечкин', tag=OpencorporaTag('NOUN,anim,masc,Sgtm,Surn sing,nomn'), normal_form='овечкин', score=1.0, methods_stack=((DictionaryAnalyzer(), 'овечкин', 37, 0),))]
[Parse(word='пожертвовал', tag=OpencorporaTag('VERB,perf,tran masc,sing,past,indc'), normal_form='пожертвовать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'пожертвовал', 748, 1),))]
[Parse(word='детской', tag=OpencorporaTag('ADJF femn,sing,gent'), normal_form='детский', score=0.394736, methods_stack=((DictionaryAnalyzer(), 'детской', 16, 8),)), Parse(word='детской', tag=OpencorporaTag('ADJF femn,sing,loct'), normal_form='детский', score=0.263157, methods_stack=((DictionaryAnalyzer(), 'детской', 16, 13),)), Parse(word='детской', tag=OpencorporaTag('ADJF femn,sing,ablt'), normal_form='детский', score=0.131578, methods_stack=((DictionaryAnalyzer(), 'детской', 16, 11),)), Parse(word='детской', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='детская', score=0.078947, methods_stack=((DictionaryAnalyz

### 2. Применить word2vec

In [76]:
# Fill all words for train

words_arr = []

for vec in data_tokens:
    words = []
    for sentence in vec[2]:
        for word in sentence:
            token = morph.parse(word)[0].normal_form  # Take the first instance
            if token not in stopwords.words("russian"):
                words.append(token)
    words_arr.append(words)

word2vec = Word2Vec(words_arr, min_count=2,vector_size=50)
vocabulary = word2vec.wv.index_to_key
print("Размер словаря:",len(vocabulary))

sim_words = word2vec.wv.most_similar("россия")
print(sim_words)


Размер словаря: 8431
[('украина', 0.9993064999580383), ('бывший', 0.9992741346359253), ('российский', 0.9992659091949463), ('умереть', 0.9992592334747314), ('клуб', 0.9992372393608093), ('сша', 0.9992058277130127), ('найти', 0.9991931915283203), ('представить', 0.999170184135437), ('оказаться', 0.9991665482521057), ('интернет', 0.9991496205329895)]


### 3. Векторизация предложений

In [77]:
v1 = word2vec.wv["россия"]
print(v1.shape)


(50,)


In [78]:
sentences_vec = []

for sentence in words_arr:
    try:
        vec = word2vec.wv[sentence[0]]
    except:
        pass
    for i in range(1,len(sentence)):
        try:
            vec = vec + word2vec.wv[sentence[i]]
        except:
            pass
    sentences_vec.append(vec)

print(sentences_vec[0])

[-0.41860983  0.29179952  0.07225689 -0.07121758 -0.31129956 -1.1069416
  1.4046897   2.1987472  -1.5065944  -0.42231905 -0.3255589  -1.0652158
 -0.21291277  0.6928413  -1.0033861   0.05026566  0.8463753  -0.35104603
 -1.6473411  -1.2871469   0.48821086  1.1424234   2.1425054  -1.1257832
  0.8041436   0.4309289  -0.8419055   0.37538263 -1.5419481   0.6640646
 -0.07586268  0.24870017 -0.30461776  0.42159936 -0.9463336   1.2704792
  0.6442732   0.26367396  0.45819473 -0.34234995  0.8988328  -0.29495433
 -0.7231193   0.11126491  2.540054    0.07222302  0.16004148 -1.3178422
  1.0574048   1.0542439 ]


In [79]:
y_uniq = []
y = []
for vec in data_tokens:
    category = vec[0]
    if y_uniq.count(category) == 0:
        y_uniq.append(category)
    y.append(y_uniq.index(category))

print(len(y_uniq))
print(y[:10])
print(len(y))

10
[0, 1, 2, 0, 3, 0, 4, 1, 4, 4]
15000


#### Настроим SVM

In [84]:
clf = svm.SVC(decision_function_shape='ovr',probability=True)
clf.fit(sentences_vec, y)

SVC(probability=True)

#### Загрузим тестовое множество

In [85]:
data_test = []

# Read Text
with open("data/news_test.txt", 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        vec = []
        category, topic, content = line.split("\t")
        vec.append(category)
        vec.append(topic)
        vec.append(content)
        data_test.append(vec)

# Tokenize


data_tokens_test = []


for vec in data_test:
    vec_tokenize = [vec[0]]
    sentences = sent_tokenize(vec[1])
    vec_tokenize.append([word_tokenize(sent) for sent in sentences])
    sentences = sent_tokenize(vec[1])
    vec_tokenize.append([word_tokenize(sent) for sent in sentences])
    data_tokens_test.append(vec_tokenize)
    
   


words_arr_test = []

for vec in data_tokens_test:
    words = []
    for sentence in vec[2]:
        for word in sentence:
            token = morph.parse(word)[0].normal_form  # Take the first instance
            if token not in stopwords.words("russian"):
                words.append(token)
    words_arr_test.append(words)    


#### Посчитаем вероятности по категориям

In [87]:
sentences_vec_test = []

for sentence in words_arr_test:
    try:
        vec = word2vec.wv[sentence[0]]
    except:
        pass
    for i in range(1,len(sentence)):
        try:
            vec = vec + word2vec.wv[sentence[i]]
        except:
            pass
    sentences_vec_test.append(vec)

dec = clf.predict_proba(sentences_vec_test)

print(dec.shape)
print(dec[0])

(3000, 10)
[0.24419811 0.21459631 0.06286252 0.12742638 0.11750383 0.04703241
 0.09037877 0.01474793 0.02948599 0.05176775]


#### Посмотрим процент попаданий

In [88]:
counter = 0


for i in range(len(dec)):
    decision = dec[i]
    index = np.where(decision == max(decision))[0][0]
    if index == y_uniq.index(data_tokens_test[i][0]):
        counter+=1
        
print(counter, "/", len(dec))

838 / 3000


Полимеальное ядро уменьшало процент попаданий. А увеличение размеронсти вектора до 100 или 500, никак не влияло.

#### Попробуем Наивный Байевский Классификатор

In [113]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [114]:
gnb = CategoricalNB()

gnb.fit(NormalizeData(sentences_vec)  , y)

CategoricalNB()

#### Предскажем результат

In [116]:
dec = gnb.predict(NormalizeData(sentences_vec_test))

#### Проверим результат

In [117]:
counter = 0


for i in range(len(dec)):
    decision = dec[i]
    if decision == y_uniq.index(data_tokens_test[i][0]):
        counter+=1
        
print(counter, "/", len(dec))

423 / 3000
